# Подготовка данных

In [1]:
import sqlite3

import pandas as pd
import numpy as np
import ta

Загрузка данных из БД созданной заранее (pj2)

In [2]:
# Установить соединение с базой данных
conn = sqlite3.connect(r'C:\Users\Alkor\gd\data_quote_db\BR_futures_minute.db')

# SQL-запрос для извлечения данных
# query = "SELECT TRADEDATE, VOLUME FROM Minute"
query = "SELECT * FROM Minute"

# Загрузить данные в DataFrame
df = pd.read_sql_query(query, conn)

# Закрыть соединение
conn.close()

# Прописные в названиях колонок
df.columns = map(str.lower, df.columns)

df

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,MXH5_2015,142000.0,141225.0,143900.0,142000.0,76,2015-03-16
1,2015-01-05 10:01:00,MXH5_2015,141700.0,141000.0,141950.0,141000.0,45,2015-03-16
2,2015-01-05 10:02:00,MXH5_2015,141000.0,140200.0,141375.0,141375.0,60,2015-03-16
3,2015-01-05 10:03:00,MXH5_2015,141275.0,141175.0,141950.0,141175.0,26,2015-03-16
4,2015-01-05 10:04:00,MXH5_2015,141075.0,140900.0,141175.0,140900.0,13,2015-03-16
...,...,...,...,...,...,...,...,...
1790342,2024-12-30 23:45:00,MXH5,300725.0,300575.0,300725.0,300725.0,74,2025-03-20
1790343,2024-12-30 23:46:00,MXH5,300725.0,300675.0,300750.0,300675.0,25,2025-03-20
1790344,2024-12-30 23:47:00,MXH5,300675.0,300600.0,300775.0,300750.0,60,2025-03-20
1790345,2024-12-30 23:48:00,MXH5,300725.0,300725.0,301000.0,300925.0,92,2025-03-20


При просмотре данных заметил, что есть мусорные данные в начале торгового дня где open, high, low, close равны друг другу.  
Проверка таких данных.

In [3]:
# Преобразуем 'tradedate' в datetime формат, если это ещё не сделано
df['tradedate'] = pd.to_datetime(df['tradedate'])

# Фильтрация по условиям
filtered_df = df[
    # Все значения равны друг другу
    (df['open'] == df['high']) & 
    (df['open'] == df['low']) &
    (df['open'] == df['close']) &
    # Минуты равны 59
    (df['tradedate'].dt.minute == 59) &  
    # Первая строка за дату
    (df.groupby(df['tradedate'].dt.date)['tradedate'].rank(method='first') == 1)  
]

filtered_df

,tradedate,secid,open,low,high,close,volume,lsttrade
1496205,2023-07-10 08:59:00,MXU3,283000.0,283000.0,283000.0,283000.0,74,2023-09-21
1497008,2023-07-11 08:59:00,MXU3,286100.0,286100.0,286100.0,286100.0,3,2023-09-21
1497749,2023-07-12 08:59:00,MXU3,286000.0,286000.0,286000.0,286000.0,29,2023-09-21
1498598,2023-07-13 08:59:00,MXU3,291500.0,291500.0,291500.0,291500.0,24,2023-09-21
1500129,2023-07-17 08:59:00,MXU3,291375.0,291375.0,291375.0,291375.0,64,2023-09-21
...,...,...,...,...,...,...,...,...
1786323,2024-12-25 09:59:00,MXH5,281000.0,281000.0,281000.0,281000.0,123,2025-03-20
1787132,2024-12-26 09:59:00,MXH5,288025.0,288025.0,288025.0,288025.0,314,2025-03-20
1787939,2024-12-27 09:59:00,MXH5,288000.0,288000.0,288000.0,288000.0,115,2025-03-20
1788746,2024-12-28 09:59:00,MXH5,288700.0,288700.0,288700.0,288700.0,4,2025-03-20


Это премаркет который потом помешает при конвертации в 5 минутные бара. Удаляем эти строки.

In [4]:
df = (
    df.merge(filtered_df, how='outer', indicator=True)
    .query('_merge == "left_only"').drop(columns=['_merge'])
    )

df

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,MXH5_2015,142000.0,141225.0,143900.0,142000.0,76,2015-03-16
1,2015-01-05 10:01:00,MXH5_2015,141700.0,141000.0,141950.0,141000.0,45,2015-03-16
2,2015-01-05 10:02:00,MXH5_2015,141000.0,140200.0,141375.0,141375.0,60,2015-03-16
3,2015-01-05 10:03:00,MXH5_2015,141275.0,141175.0,141950.0,141175.0,26,2015-03-16
4,2015-01-05 10:04:00,MXH5_2015,141075.0,140900.0,141175.0,140900.0,13,2015-03-16
...,...,...,...,...,...,...,...,...
1790342,2024-12-30 23:45:00,MXH5,300725.0,300575.0,300725.0,300725.0,74,2025-03-20
1790343,2024-12-30 23:46:00,MXH5,300725.0,300675.0,300750.0,300675.0,25,2025-03-20
1790344,2024-12-30 23:47:00,MXH5,300675.0,300600.0,300775.0,300750.0,60,2025-03-20
1790345,2024-12-30 23:48:00,MXH5,300725.0,300725.0,301000.0,300925.0,92,2025-03-20


# Конвертирование 1 минутных котировок в 5 минутные

In [5]:
# Убедимся, что колонка  tradedate имеет тип datetime
df["tradedate"] = pd.to_datetime(df["tradedate"])

# Устанавливаем  tradedate как индекс
df.set_index("tradedate", inplace=True)

# Агрегирование до 5-минутных баров
df_m5 = df.resample("5min").agg({
    "open": "first",    # Первая цена в периоде
    "high": "max",      # Максимальная цена
    "low": "min",       # Минимальная цена
    "close": "last",    # Последняя цена
    "volume": "sum"    # Суммарный объём
})

# Сбрасываем индекс
df_m5 = df_m5.reset_index()

# Стереть строки с NaN
df_m5.dropna(inplace=True)

# Переиндексация
df_m5 = df_m5.reset_index(drop=True)

df_m5

,tradedate,open,high,low,close,volume
0,2015-01-05 10:00:00,142000.0,143900.0,140200.0,140900.0,220
1,2015-01-05 10:05:00,140700.0,141000.0,138525.0,140600.0,91
2,2015-01-05 10:10:00,140300.0,141000.0,140200.0,140650.0,23
3,2015-01-05 10:15:00,140200.0,141050.0,140200.0,141050.0,47
4,2015-01-05 10:20:00,141000.0,141500.0,140825.0,140900.0,19
...,...,...,...,...,...,...
407824,2024-12-30 23:25:00,301125.0,301350.0,300400.0,300450.0,621
407825,2024-12-30 23:30:00,300450.0,300525.0,300400.0,300500.0,236
407826,2024-12-30 23:35:00,300450.0,300450.0,300275.0,300425.0,152
407827,2024-12-30 23:40:00,300450.0,300750.0,300425.0,300700.0,230


### TVI

In [6]:
def calculate_ema(series, period):
    """
    Вычисляет экспоненциальное скользящее среднее (EMA) 
    для указанного ряда данных.
    """
    return series.ewm(span=period, adjust=False).mean()

def add_tvi_column(df, r=12, s=12, u=5, point=0.0001):
    """
    Добавляет колонку TVI в DataFrame.
    
    :param df: DataFrame с колонками ['open', 'close', 'volume']
    :param r: Период для EMA для UpTicks и DownTicks
    :param s: Период для второго EMA
    :param u: Период для EMA на TVI_calculate
    :param point: Минимальное изменение цены (параметр MyPoint в MQL4)
    :return: DataFrame с добавленной колонкой 'TVI'
    """
    # Проверяем, что необходимые колонки существуют
    required_columns = ['open', 'close', 'volume']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"DataFrame должен содержать колонку '{col}'")

    # Вычисляем UpTicks и DownTicks
    df['UpTicks'] = (df['volume'] + (df['close'] - df['open']) / point) / 2
    df['DownTicks'] = df['volume'] - df['UpTicks']

    # EMA для UpTicks и DownTicks
    df['EMA_UpTicks'] = calculate_ema(df['UpTicks'], r)
    df['EMA_DownTicks'] = calculate_ema(df['DownTicks'], r)

    # Второе EMA для UpTicks и DownTicks
    df['DEMA_UpTicks'] = calculate_ema(df['EMA_UpTicks'], s)
    df['DEMA_DownTicks'] = calculate_ema(df['EMA_DownTicks'], s)

    # Расчет TVI_calculate
    df['TVI_calculate'] = 100.0 * (df['DEMA_UpTicks'] - df['DEMA_DownTicks']) / (
        df['DEMA_UpTicks'] + df['DEMA_DownTicks']
    )

    # EMA на TVI_calculate
    df['TVI'] = calculate_ema(df['TVI_calculate'], u)

    # Удаляем промежуточные колонки, если они не нужны
    df.drop(
        columns=['UpTicks', 'DownTicks', 'EMA_UpTicks', 'EMA_DownTicks', 
        'DEMA_UpTicks', 'DEMA_DownTicks', 'TVI_calculate'], inplace=True
        )

    return df

# Добавляем колонку TVI
df_m5 = add_tvi_column(df_m5, r=12, s=12, u=5, point=0.1)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI
0,2015-01-05 10:00:00,142000.0,143900.0,140200.0,140900.0,220,-5000.000000
1,2015-01-05 10:05:00,140700.0,141000.0,138525.0,140600.0,91,-4987.089970
2,2015-01-05 10:10:00,140300.0,141000.0,140200.0,140650.0,23,-4951.385575
3,2015-01-05 10:15:00,140200.0,141050.0,140200.0,141050.0,47,-4864.419074
4,2015-01-05 10:20:00,141000.0,141500.0,140825.0,140900.0,19,-4757.102148
...,...,...,...,...,...,...,...
407824,2024-12-30 23:25:00,301125.0,301350.0,300400.0,300450.0,621,90.993674
407825,2024-12-30 23:30:00,300450.0,300525.0,300400.0,300500.0,236,87.899935
407826,2024-12-30 23:35:00,300450.0,300450.0,300275.0,300425.0,152,80.898785
407827,2024-12-30 23:40:00,300450.0,300750.0,300425.0,300700.0,230,78.349127


## GannHiLo

In [7]:
def calculate_sma(series, period):
    """
    Вычисляет простую скользящую среднюю (SMA).
    """
    return series.rolling(window=period, min_periods=1).mean()

def add_gann_hilo_column_optimized(df, period=10):
    """
    Оптимизированная версия добавления индикатора GannHiLo в DataFrame.
    
    :param df: DataFrame с колонками ['close', 'high', 'low']
    :param period: Период для расчета SMA
    :return: DataFrame с добавленными колонками 'Gann_HiLo'
    """
    # Проверка на необходимые колонки
    required_columns = ['close', 'high', 'low']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(
            f"DataFrame должен содержать колонки: {required_columns}"
            )

    # Расчет SMA для high и low
    sma_high = calculate_sma(df['high'], period)
    sma_low = calculate_sma(df['low'], period)

    # Вычисление GannHiLo векторным способом
    gann_hilo = (
        (df['close'] > sma_high.shift(1)).astype(int) - 
        (df['close'] < sma_low.shift(1)).astype(int)
        )

    # Добавляем колонку Gann_HiLo в DataFrame
    df['Gann_HiLo'] = gann_hilo

    return df

# Добавляем GannHiLo индикатор
df_m5 = add_gann_hilo_column_optimized(df_m5, period=10)

# Результат
df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo
0,2015-01-05 10:00:00,142000.0,143900.0,140200.0,140900.0,220,-5000.000000,0
1,2015-01-05 10:05:00,140700.0,141000.0,138525.0,140600.0,91,-4987.089970,0
2,2015-01-05 10:10:00,140300.0,141000.0,140200.0,140650.0,23,-4951.385575,0
3,2015-01-05 10:15:00,140200.0,141050.0,140200.0,141050.0,47,-4864.419074,0
4,2015-01-05 10:20:00,141000.0,141500.0,140825.0,140900.0,19,-4757.102148,0
...,...,...,...,...,...,...,...,...
407824,2024-12-30 23:25:00,301125.0,301350.0,300400.0,300450.0,621,90.993674,0
407825,2024-12-30 23:30:00,300450.0,300525.0,300400.0,300500.0,236,87.899935,0
407826,2024-12-30 23:35:00,300450.0,300450.0,300275.0,300425.0,152,80.898785,0
407827,2024-12-30 23:40:00,300450.0,300750.0,300425.0,300700.0,230,78.349127,0


## T3 (T3ma) (8, 0.618)

In [8]:
def calculate_t3ma(df, period=8, b=0.618):
    """
    Рассчитывает индикатор T3MA и добавляет его как новую колонку в DataFrame.

    :param df: DataFrame с колонкой 'close'
    :param period: Период T3MA
    :param b: Параметр T3MA (значение сглаживания)
    :return: DataFrame с добавленной колонкой 'T3MA'
    """
    # Проверка на наличие колонки 'close'
    if 'close' not in df.columns:
        raise ValueError("DataFrame должен содержать колонку 'close'")

    close = df['close'].values

    # Коэффициенты
    b2 = b ** 2
    b3 = b2 * b
    c1 = -b3
    c2 = 3 * (b2 + b3)
    c3 = -3 * (2 * b2 + b + b3)
    c4 = 1 + 3 * b + b3 + 3 * b2
    n = max(1, 1 + 0.5 * (period - 1))
    w1 = 2 / (n + 1)
    w2 = 1 - w1

    # Создание временных массивов
    e1 = np.zeros_like(close)
    e2 = np.zeros_like(close)
    e3 = np.zeros_like(close)
    e4 = np.zeros_like(close)
    e5 = np.zeros_like(close)
    e6 = np.zeros_like(close)

    # Векторизованный расчет экспоненциальных сглаживаний
    e1[0] = close[0]
    for i in range(1, len(close)):
        e1[i] = w1 * close[i] + w2 * e1[i - 1]

    e2[0] = e1[0]
    for i in range(1, len(e1)):
        e2[i] = w1 * e1[i] + w2 * e2[i - 1]

    e3[0] = e2[0]
    for i in range(1, len(e2)):
        e3[i] = w1 * e2[i] + w2 * e3[i - 1]

    e4[0] = e3[0]
    for i in range(1, len(e3)):
        e4[i] = w1 * e3[i] + w2 * e4[i - 1]

    e5[0] = e4[0]
    for i in range(1, len(e4)):
        e5[i] = w1 * e4[i] + w2 * e5[i - 1]

    e6[0] = e5[0]
    for i in range(1, len(e5)):
        e6[i] = w1 * e5[i] + w2 * e6[i - 1]

    # Итоговый расчет T3
    t3 = c1 * e6 + c2 * e5 + c3 * e4 + c4 * e3

    # Добавление результата в DataFrame
    df['T3MA'] = t3
    return df

# Рассчитываем T3MA и добавляем в DataFrame
df_m5 = calculate_t3ma(df_m5, period=8, b=0.618)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI,Gann_HiLo,T3MA
0,2015-01-05 10:00:00,142000.0,143900.0,140200.0,140900.0,220,-5000.000000,0,140900.000000
1,2015-01-05 10:05:00,140700.0,141000.0,138525.0,140600.0,91,-4987.089970,0,140860.985003
2,2015-01-05 10:10:00,140300.0,141000.0,140200.0,140650.0,23,-4951.385575,0,140805.018055
3,2015-01-05 10:15:00,140200.0,141050.0,140200.0,141050.0,47,-4864.419074,0,140802.000144
4,2015-01-05 10:20:00,141000.0,141500.0,140825.0,140900.0,19,-4757.102148,0,140820.920747
...,...,...,...,...,...,...,...,...,...
407824,2024-12-30 23:25:00,301125.0,301350.0,300400.0,300450.0,621,90.993674,0,300597.497660
407825,2024-12-30 23:30:00,300450.0,300525.0,300400.0,300500.0,236,87.899935,0,300622.068191
407826,2024-12-30 23:35:00,300450.0,300450.0,300275.0,300425.0,152,80.898785,0,300605.154016
407827,2024-12-30 23:40:00,300450.0,300750.0,300425.0,300700.0,230,78.349127,0,300603.834381


# CCI (20)

In [9]:
# Вычисление CCI с периодом 20
df_m5['CCI_20'] = ta.trend.cci(
    high=df_m5['high'], low=df_m5['low'], close=df_m5['close'], window=20
    )

df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo,T3MA,CCI_20
0,2015-01-05 10:00:00,142000.0,143900.0,140200.0,140900.0,220,-5000.000000,0,140900.000000,NaN
1,2015-01-05 10:05:00,140700.0,141000.0,138525.0,140600.0,91,-4987.089970,0,140860.985003,NaN
2,2015-01-05 10:10:00,140300.0,141000.0,140200.0,140650.0,23,-4951.385575,0,140805.018055,NaN
3,2015-01-05 10:15:00,140200.0,141050.0,140200.0,141050.0,47,-4864.419074,0,140802.000144,NaN
4,2015-01-05 10:20:00,141000.0,141500.0,140825.0,140900.0,19,-4757.102148,0,140820.920747,NaN
...,...,...,...,...,...,...,...,...,...,...
407824,2024-12-30 23:25:00,301125.0,301350.0,300400.0,300450.0,621,90.993674,0,300597.497660,139.374669
407825,2024-12-30 23:30:00,300450.0,300525.0,300400.0,300500.0,236,87.899935,0,300622.068191,64.516129
407826,2024-12-30 23:35:00,300450.0,300450.0,300275.0,300425.0,152,80.898785,0,300605.154016,37.865236
407827,2024-12-30 23:40:00,300450.0,300750.0,300425.0,300700.0,230,78.349127,0,300603.834381,84.282567


# Сигналы индикаторов

In [10]:
# Добавляем новую колонку с сигналом TVI
df_m5['TVI_S'] = df_m5['TVI'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Добавляем новую колонку с сигналом T3MA
df_m5['T3_S'] = df_m5['T3MA'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Добавляем новую колонку с сигналом от CCI
df_m5['CCI_S'] = df_m5['CCI_20'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

df_m5 = df_m5[['tradedate', 'open', 'high', 'low', 'close', 
               'TVI_S', 'CCI_S', 'T3_S', 'Gann_HiLo']]

df_m5 = df_m5.copy()

# Переименование колонок
df_m5.rename(
    columns={'TVI_S': 'tvi', 'CCI_S': 'cci', 'T3_S': 't3', 'Gann_HiLo': 'ghl'}, 
    inplace=True
    )

df_m5

,tradedate,open,high,low,close,tvi,cci,t3,ghl
0,2015-01-05 10:00:00,142000.0,143900.0,140200.0,140900.0,0,0,0,0
1,2015-01-05 10:05:00,140700.0,141000.0,138525.0,140600.0,1,0,-1,0
2,2015-01-05 10:10:00,140300.0,141000.0,140200.0,140650.0,1,0,-1,0
3,2015-01-05 10:15:00,140200.0,141050.0,140200.0,141050.0,1,0,-1,0
4,2015-01-05 10:20:00,141000.0,141500.0,140825.0,140900.0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
407824,2024-12-30 23:25:00,301125.0,301350.0,300400.0,300450.0,1,1,1,0
407825,2024-12-30 23:30:00,300450.0,300525.0,300400.0,300500.0,-1,1,1,0
407826,2024-12-30 23:35:00,300450.0,300450.0,300275.0,300425.0,-1,1,-1,0
407827,2024-12-30 23:40:00,300450.0,300750.0,300425.0,300700.0,-1,1,-1,0


### Сохранение в файл

In [11]:
df_m5.to_csv("br_m5_TVI_CCI_T3_GHL.csv", index=False)